# Chapter 7 Efficiency

미국의 컴퓨터 과학자인 Alan Perlis가 애기하기를...

> A functional programmer was someone who knew the value of everything and the cost of nothing.

functional programming의 **성능**과 **효율성**은?








## Markdown
You can use [Markdown syntax](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet)
for every Markdown cell. This allows you to embed
- headers,
- lists,
- images,
- and so on.

## Mathematics
In addition to standard Markdown, you can use LaTeX-flavored math:

```tex
\begin{align*}
    a &= b + c(z) \\
    f &= \nabla \cdot a^2
\end{align*}
```
\begin{align*}
    a &= b + c(z) \\
    f &= \nabla \cdot a^2
\end{align*}

<hr/>

```tex
We now compute $a + b = c$.
```
We now compute $a + b = c$.

<hr/>

```tex
Sometimes, you want equations:
$$a + b = \nabla^2 c.$$
```
Sometimes, you want equations:
$$a + b = \nabla^2 c.$$

# 7.1 Lazy Evaluation


## Lazy

    sqr (sqr (3+4))

<s>위 expression에 대해서 아래와 같이 lazy하게 reduction 된다.</s>

    sqr (sqr (3+4))
    = sqr (3+4) * sqr (3+4)
    = ((3+4)*(3+4)) * ((3+4)*(3+4))
    = ...
    = 2401

lazy evaluation은 아래와 같이 reduction된다.

    sqr (sqr (3+4))
    = let x = sqr (3+4) in sqr x
    = let y = 3+4 in
      let x = sqr y in sqr x
    = let y = 7 in
      let x = sqr y in sqr x
    = let x = sqr 7 in sqr x
    = let x = 7 * 7 in sqr x
    = let x = 49 in sqr x
    = 49 * 49
    = 2401


> <s>Lazy evaluation arguments는 필요한 시점에 단 한 번만 evaluate 된다.</s>

보다 정확하게 얘기하면

> Lazy evaluation에서 arguments는 **필요한 시점**에 단 **한 번만** **head normal form** 으로 evaluate 된다.

## Head Normal Form

(Weak) Head Normal Form은 top-level이 아래로 구성되어 있다.

 * data constructor
 * fully reduced lambda abstraction (weak: any lambda abstraction)

NF는 HNF이나 HNF가 NF는 아니다.

 * NF: 42, (2,"hello"), \x -> (x+1), e1, e2
 * HNF: (1+2,2+3), \x -> 2+2, 'h':("e"++"llo"), x, (e1, e2)
 * ???: 1+1, (\x -> x+1) 2

'sqr (head xs)'의 evaluation

    sqr (head xs)
    = let a = head xs in sqr a
    = let b = xs in
      let a = head xs in sqr a
    = let b = y:ys in
      let a = head xs in sqr a
    = let a = head (y:ys) in sqr a
    = let a = y in sqr a
    = sqr y
    = y * y

## Common Subexpression elimination

    subseqs (x:xs) = subseqs xs ++ map (x:) (subseqs xs)
    subseqs' (x:xs) = xss ++ map (x:) xss
                      where xss = subseqs' xs

subseqs는 'subseqs xs'가 2번 evaluate된다. 이것을 where로 분리하는 경우 1번만 연산할 수 있으나 **haskell은 common subexpression elimination 을 자동으로 하지 않는다.**
추가 공간을 사용하여 시간을 최적화하는 것이므로 프로그래머가 결정해야하는 사항임.

## Binding

    foo1 n = sum (take n primes)
      where
      primes = [x | x <- [2..], divisors x == [x]]
      divisors x = [d | d <- [2..x], x `mod` d == 0]
    
    foo2 n = sum (take n primes)
    primes = [x | x <- [2..], divisors x == [x]]
    divisors x = [d | d <- [2..x], x `mod` d == 0]
    
    foo3 = \n -> sum (take n primes)
      where
      primes = [x | x <- [2..], divisors x == [x]]
      divisors x = [d | d <- [2..x], x `mod` d == 0]

primes, divisors의 binding

 * foo1: 'foo1 n'에 bind 됨
 * foo2: global에 bind 됨
 * foo3: 'foo3'에 bind 됨

foo2, foo3의 경우 evaluation 결과를 재사용하나 공간 사용량이 증가함

# 7.2. Controlling space

## lazy and space

lazy evaluation

    sum [1..1000]
    = foldl (+) 0 [1..1000]
    = foldl (+) (0+1) [2..1000]
    = foldl (+) ((0+1)+2) [3..1000]
    = ...
    = 500500

eager evaluation

    sum [1..1000]
    = foldl (+) 0 [1..1000]
    = foldl (+) (0+1) [2..1000]
    = foldl (+) 1 [2..1000]
    = foldl (+) (1+2) [3..1000]
    = ...
    = 500500

**space를 제어하기 위해서는 eager evaluation을 lazy evaluation과 함께 사용하는 것이 좋다.**

## eager evaluation

    seq :: a -> b -> b

    -- Data.List.foldl'
    foldl' :: (b -> a -> b) -> b -> [a] -> b
    foldl' f e [] = e
    foldl' f e (x:xs) = y `seq` foldl' f y xs
                        where y = f e x

Data\.List\.sum과 foldl'은 위 방식으로 구현됨.

lazy와 eager 구현은 strict function에 대해서는 동일하다. (f &perp; = &perp; 라면, f는 strict하다.)

## mean 예제

    mean [] = 0
    mean xs = sum xs / fromIntegral (length xs)

xs의 순회가 2번 발생한다. tupling을 사용하여 1번 순회로 합과 길이를 구함.

    sumlen :: [Float] -> (Float,Int)
    sumlen = foldr f (0,0)
             where f x (s,n) = (s+x,n+1)
    
    mean [] = 0
    mean xs = s / fromIntegral n
         where (s,n) = sumlen xs

space leak을 제거하기 위해서 foldl'을 사용

    sumlen = foldl' g (0,0)
             where g (s,n) x = (s+x,n+1)

HNF인 (s+x,n+1)에 eager evaluation 적용

    sumlen = foldl' f (0,0)
             where f (s,n) x = s `seq` n `seq` (s+x,n+1)

two more application operators

 * $: lazy evaluation
 * \$\!: eager evaluation

# 7.3 Controlling time

## tips for time

eager evalution을 사용하여 쉽게 제어할 수 있으나 속도는 그렇지 못 하다.

GHC 문서가 제시하는 속도향상의 키 포인트 3가지

1. GHC Profiling 도구를 사용해라
2. 알고리즘을 개선해라
3. 가능하면 제공되는 라이브러리를 사용하라. (매우 최적화되어 있고, 컴파일되어 있음.)

추가적인 팁 2가지

* 'Strict funcions are your dear friends': 보통 eager evaluation이 lazy evaluation보다 overhead가 적다.
* 요구사항을 만족하는 최적의 type을 명시적으로 사용할 것: Integer보다는 Int가 좋다.

이러한 방법들은 asymptotic time complexity을 변환시키지 못 하므로 효과가 작다. 그러나 잘못된 코드는 asymptotic complexity에 영향을 줄 수도 있다.

## cartesian product

    cp [] = [[]]
    cp (xs:xss) = [x:ys | x <- xs, ys <- cp xss]

    cp' = foldr op [[]]
          where op xs yss = [x:ys | x <- xs, ys <- yss]

cp보다 cp'의 evaluation time이 훨씬 적다. cp는 cp xss를 xs의 길이만큼 반복 계산한다.

    cp [] = [[]]
    cp (xs:xss) = concat (map f xs)
                  where f x = [x:ys | ys <- cp xss]

아래 cp''는 cp'와 동일한 성능을 가진다.

    cp'' [] = [[]]
    cp'' (x:xss) = [x:ys | x <- xs, ys <- yss]
                   where yss = cp xss

# 7.4 Analysing time

# 7.5 Accumulating Parameter

# 7.6 Tupling

# 7.7 Sorting


# Reference

* Profiling
 * https://downloads.haskell.org/~ghc/latest/docs/html/users_guide/profiling.html
 * http://book.realworldhaskell.org/read/profiling-and-optimization.html

